In [1]:
library(nnet)
library(party)
library(randomForest)
library(e1071)
library(caret)

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘ggplot2’

The following object is masked from ‘package:randomForest’:

    margin



In [2]:
# Sampling
iris2 <- as.data.frame(scale(iris[,-5]))
iris2$Species <- iris$Species
samp <- createDataPartition(iris2$Species, p=0.7, list=F)
# 학습용
data.tr <- iris2[samp,]
# 검증용
data.test <- iris2[-samp,]

# 문제 및 정답 만들기
x <- subset(data.test, select=-Species)
y <- data.test$Species

In [3]:
# 인공신경망
nnetF <- function(q){
  result = 0
  for(i in 1:q){
    # 학습하기
    model.nnet <- nnet(Species~.,data=data.tr, size=3) 

    # 검증하기
    pred <- predict(model.nnet, x, type='class')

    # 예측율  
    result = (result + mean(pred == y))
  }
  return(result/q)
}

# 의사결정나무
ctreeF <- function(q){
  result = 0
  for(i in 1:q){
  # 학습하기
  model.ctree <- ctree(Species~.,data=data.tr)

  # 검증하기
  pred <- predict(model.ctree, x)

  # 예측율
  result <- (result + mean(pred == y))
  }
  return(result/q)
}

# Random Forest
rfF <- function(q){  
  result = 0
  for(i in 1:q){
  # 학습하기
  model.rf <- randomForest(Species~., data=data.tr, ntree=100)

  # 검증하기
  pred <- predict(model.rf, x)

  # 예측율  
  result <- (result + mean(pred == y))
  }
  return(result/q)
}

# SVM
svmF <- function(q){  
  result = 0
  for(i in 1:q){
  # 학습하기
  model.svm = svm(Species~., data=data.tr)

  # 검증하기
  pred <- predict(model.svm, x)

  # 예측율
  result <- (result + mean(pred == y))
  }
  return(result/q)
}

# 다항 로지스틱 회귀분석
mulF <- function(q){  
  result = 0
  for(i in 1:q){
  # 학습하기
  model.mul <- multinom(Species~., data=data.tr)

  # 검증하기
  pred <- predict(model.mul, x)

  # 예측율  
  result <- (result + mean(pred == y))
  }
  return(result/q)
}


In [6]:
final <- function(q){
  result <- data.frame(예측력=c(nnetF(q),ctreeF(q),rfF(q),svmF(q),mulF(q)))
  rownames(result) <- c('인공신경망','의사결정나무','RandomForest','SVM','로지스틱회귀분석')
  return(result)
}  

In [8]:
final(5)

# weights:  27
initial  value 107.906747 
iter  10 value 7.561239
iter  20 value 5.240224
iter  30 value 4.189245
iter  40 value 0.446212
iter  50 value 0.001571
final  value 0.000056 
converged
# weights:  27
initial  value 117.994663 
iter  10 value 7.749043
iter  20 value 5.423663
iter  30 value 4.846408
iter  40 value 1.255683
iter  50 value 0.233693
iter  60 value 0.046497
iter  70 value 0.007565
iter  80 value 0.003143
iter  90 value 0.002667
iter 100 value 0.002015
final  value 0.002015 
stopped after 100 iterations
# weights:  27
initial  value 118.725125 
iter  10 value 9.247711
iter  20 value 5.277430
iter  30 value 4.596861
iter  40 value 4.126815
iter  50 value 4.004259
iter  60 value 3.997117
iter  70 value 3.995097
iter  80 value 3.986558
iter  90 value 3.982680
iter 100 value 3.978685
final  value 3.978685 
stopped after 100 iterations
# weights:  27
initial  value 128.293829 
iter  10 value 7.771173
iter  20 value 4.647860
iter  30 value 4.202851
iter  40 value 4.103463

,예측력
인공신경망,0.9111111
의사결정나무,0.8888889
RandomForest,0.9288889
SVM,0.9555556
로지스틱회귀분석,0.9777778
